# Необходимо поднять RAG-pipeline на базе конспектов.

В качестве базы конспектов были выбраны викиконспекты от университета ИТМО по предмету "Алгоритмы и структуры данных". По информации об их crawling'е обратите внимание на ноутбук `DBFiller.ipynb`.

## Подготовка

In [1]:
# Run if exists
!unzip -q chroma_db.zip

Установим зависимости

In [1]:
%pip install requests pypdf tiktoken
%pip install langchain langchain-chroma chromadb
%pip install ollama langchain-community
%pip install docker

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Запустим локальную модель для начала

In [2]:
!/usr/bin/bash up_ollama.sh
import docker


def is_container_running(container_name: str):
    """Verify the status of a container by it's name

    :param container_name: the name of the container
    :return: boolean or None
    """
    RUNNING = "running"
    docker_client = docker.from_env()
    try:
        container = docker_client.containers.get(container_name)
    except docker.errors.NotFound as exc:
        print(f"Check container name!\n{exc.explanation}")
    else:
        container_state = container.attrs["State"]
        return container_state["Status"] == RUNNING

import time
while not is_container_running("ollama"):
    time.sleep(10)

!docker exec ollama ollama run llama3

f681a7c73fc08ef07f23d93113e442a68cafa555ecc4e1c12565b53f2d976050
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕     

У меня была модель, доступная на сервере, поэтому следующие параметры должны быть получены из переменных среды:

In [3]:
import os

model_name = "llama3"
url = "http://0.0.0.0:11434" 

## Подготовка базы данных

In [4]:
import unicodedata

ACCENT_MAPPING = {
    '́': '',
    '̀': '',
    'а́': 'а',
    'а̀': 'а',
    'е́': 'е',
    'ѐ': 'е',
    'и́': 'и',
    'ѝ': 'и',
    'о́': 'о',
    'о̀': 'о',
    'у́': 'у',
    'у̀': 'у',
    'ы́': 'ы',
    'ы̀': 'ы',
    'э́': 'э',
    'э̀': 'э',
    'ю́': 'ю',
    '̀ю': 'ю',
    'я́́': 'я',
    'я̀': 'я',
}
ACCENT_MAPPING = {unicodedata.normalize('NFKC', i): j for i, j in ACCENT_MAPPING.items()}


def unaccentify(s):
    source = unicodedata.normalize('NFKC', s)
    for old, new in ACCENT_MAPPING.items():
        source = source.replace(old, new)
    return source

def normalize(text):
    return (
        unaccentify(text)
        .replace('«','')
        .replace('»','')
        .replace('"','')
        .replace('<','')
        .replace('>','')
    )


In [5]:
with open('./text_db/hashing.txt',encoding='utf-8') as f:
    hashing = ''.join(f.readlines())
with open('./text_db/sorting.txt',encoding='utf-8') as f:
    sorting = ''.join(f.readlines())

hashing = [normalize(x) for x in hashing.split('-----')]
sorting = [normalize(x) for x in sorting.split('-----')]

In [6]:
fragment_size = 2500
words_amount = 500
fragments = set(hashing+sorting)
reduced_fragments = set()

for element in fragments:
    if len(element) > fragment_size:
        words = element.split()
        chunks = [" ".join(words[x:x + words_amount]) for x in range(0, len(words), words_amount)]
        reduced_fragments |= set(chunks)
        continue
    reduced_fragments.add(element)

reduced_fragments.remove("")

max(len(x) for x in reduced_fragments)

3885

In [7]:
len(reduced_fragments)

38

In [8]:
print(reduced_fragments)

{'Сортировка выбором (англ. selection sort) простой алгоритм сортировки со сложностью O(n^2), где n количество элементов для сортировки. Алгоритм На каждом i-ом шаге алгоритма находим i-ый минимальный элемент и меняем его местами с i-ым элементом в массиве. Таким образом будет получен массив, отсортированный по неубыванию. Псевдокод Вариант 1. Будем каждый раз проходить по всем еще не отсортированным элементам, и, как только найдем элемент меньше, чем первый из неотсортированных, поменяем их местами. Таким образом будет нужно O(n^2) обменов (для каждого i требуется O(n-i) обменов). function selectionSort(T[n] a): for i = 0 to n - 2 for j = i + 1 to n - 1 if a[i] a[j] swap(a[i], a[j]) Вариант 2. Второй вариант немного более экономный. Здесь мы будем менять местами элементы только 1 раз для каждого i, всего будет нужно O(n) обменов. Для этого сначала мы будем проходить по всем еще не отсортированным элементам, искать минимальный, и только потом менять местами минимальный и первый из неот

## Подготовка модели

In [9]:
import requests

def get_ollama_embeddings(text):
  response = requests.post(f"{url}/api/embeddings", json={
    "model": model_name,
    "prompt": text
  })

  return response.json().get("embedding")


In [10]:
from langchain_chroma import Chroma
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb.types import Vector

class MyEmbeddingFunction(EmbeddingFunction):
    @staticmethod
    def embed_documents(input: Documents) -> Embeddings:
        
        return [get_ollama_embeddings(text) for text in input]
    
    @staticmethod
    def embed_query(query: str) -> Vector:
        return get_ollama_embeddings(query)


In [11]:
import chromadb
from chromadb.errors import InvalidDimensionException

db_dirname = "./chromadb"
if not os.path.exists(db_dirname):
    chromadb.api.client.SharedSystemClient.clear_system_cache()

    try:
        db = Chroma.from_texts(list(reduced_fragments), embedding=MyEmbeddingFunction, persist_directory=db_dirname)
    except InvalidDimensionException:
        Chroma().delete_collection()
        db = Chroma.from_texts(list(reduced_fragments), embedding=MyEmbeddingFunction, persist_directory=db_dirname)
else:
    db = Chroma(embedding_function=MyEmbeddingFunction, persist_directory=db_dirname)



In [12]:
db

In [42]:
q = "Что такое совершенное хеширование?"
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 10})
res = retriever.invoke(q)
res

[Document(id='e826aaf6-03f0-4f09-b8f7-5cbf02f3bb78', metadata={}, page_content=' Введение \nСуществует два основных вида хеш-таблиц: с цепочками и открытой адресацией. Хеш-таблица содержит некоторый массив H, элементы которого есть пары (хеш-таблица с открытой адресацией) или списки пар (хеш-таблица с цепочками).\n\nВыполнение операции в хеш-таблице начинается с вычисления хеш-функции от ключа. Хеш-код i = h(key) играет роль индекса в массиве H, а зная индекс, мы можем выполнить требующуюся операцию (добавление, удаление или поиск).\n\nКоличество коллизий зависит от хеш-функции; чем лучше используемая хеш-функция, тем меньше вероятность их возникновения. \n\nСпособ разрешения коллизий — важная составляющая любой хеш-таблицы.\n\nПолностью избежать коллизий для произвольных данных невозможно в принципе, и хорошая хеш-функция в состоянии только минимизировать их количество. Но, в некоторых специальных случаях их удаётся избежать. Если все ключи элементов известны заранее, либо меняются оч

## Настройка RAG-конвейера

In [39]:
import ollama
import langchain.chains
import langchain.prompts
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

import typing as tp
from langchain_core.language_models.llms import LLM

# LLM
class CustomLlama(LLM):
    def _call(self, prompt, **kwargs):
        client = Client(
            host=url
        )
        response = client.chat(model=model_name, messages=[
            {
                'role': 'user',
                'content': prompt,
            },
        ])

        return response.message.content

    @property
    def _identifying_params(self) -> tp.Dict[str, tp.Any]:
        """Return a dictionary of identifying parameters."""
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": "CustomChatModel",
        }

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "custom"

ollama_llm = CustomLlama()

prompt = """
Ты — русскоязычный ассистент. Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста. Выведи только
краткий ответ, не надо пояснительного текста. Отвечай только на поставленный вопрос. Пожалуйста, отвечай на вопросы только на русском языке.
Текст:
-----
{context}
-----
Вопрос:
{question}"""

prompt = langchain.prompts.PromptTemplate(
    template=prompt, input_variables=["context", "question"]
)

def join_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Создаём цепочку
chain = (
    {"context": retriever | join_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama_llm
    | StrOutputParser()
)

In [43]:
q_for_sort = "Какие бывают сортировки?"
retriever_sort = db.as_retriever(search_type="mmr", search_kwargs={"k": 15})
res_for_sort = retriever_sort.invoke(q_for_sort)
res_for_sort

[Document(id='e826aaf6-03f0-4f09-b8f7-5cbf02f3bb78', metadata={}, page_content=' Введение \nСуществует два основных вида хеш-таблиц: с цепочками и открытой адресацией. Хеш-таблица содержит некоторый массив H, элементы которого есть пары (хеш-таблица с открытой адресацией) или списки пар (хеш-таблица с цепочками).\n\nВыполнение операции в хеш-таблице начинается с вычисления хеш-функции от ключа. Хеш-код i = h(key) играет роль индекса в массиве H, а зная индекс, мы можем выполнить требующуюся операцию (добавление, удаление или поиск).\n\nКоличество коллизий зависит от хеш-функции; чем лучше используемая хеш-функция, тем меньше вероятность их возникновения. \n\nСпособ разрешения коллизий — важная составляющая любой хеш-таблицы.\n\nПолностью избежать коллизий для произвольных данных невозможно в принципе, и хорошая хеш-функция в состоянии только минимизировать их количество. Но, в некоторых специальных случаях их удаётся избежать. Если все ключи элементов известны заранее, либо меняются оч

In [44]:
chain_for_sort = (
    {"context": retriever_sort | join_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama_llm
    | StrOutputParser()
)

## Тестирование

In [27]:
from ollama import AsyncClient, Client

In [28]:
result = chain.invoke(q)
print(result)

Привет! Я русскоязычный автоматический ассистент. 

Совершенное хеширование (perfect hashing) — это метод хранения и поиска данных в хеш-таблицах, при котором каждый из n ключей имеет уникальное хеш-значение, то есть не существует двух различных ключей с равными хеш-значениями. Это означает, что можно выполнить поиск ключа в хеш-таблице за постоянное время О(1), не учитывая времени на сравнение значений.

Совершенное хеширование может быть реализовано, если использовать два уровня хеширования: основное и дополнительное. Основное хеширование выбирает блоки памяти, а дополнительное — внутриблоковое хеширование выбирает конкретный элемент в выбранном блоке.

Таким образом, совершенное хеширование обеспечивает быстрый поиск данных в хеш-таблицах, что является важным приложением в различных областях, таких как компьютерная графика, базы данных и другие.


In [45]:
result = chain_for_sort.invoke('Какие бывают квадратичные сортировки?')
print(result)

Вот несколько примеров квадратичных сортировок:

1. Сортировка выбором (Selection sort) - это простой алгоритм, который работает за время O(n^2), где n - количество элементов для сортировки.
2. Пузырьковая сортировка (Bubble sort) - это еще один простой алгоритм, работающий также за время O(n^2).
3. Сортировка вставками (Insertion sort) - это алгоритм, который работает эффективнее для небольших размеров входных данных, но имеет квадратичную сложность для больших данных.
4. Шелл-сортировка (Shell sort) - это алгоритм, работающий за время O(n^2), но лучше, чем сортировка выбором и пузырьковая сортировка, при некоторых условиях.

В целом, квадратичные сортировки не рекомендуется использовать для больших размеров входных данных, поскольку они могут быть слишком медленными. Вместо этого, предпочтительно использовать более эффективные алгоритмы с линейной или логарифмической сложностью, такие как быстрая сортировка (Quick sort) или сортировка кучей (Heap sort).


In [46]:
result = chain_for_sort.invoke('Какая сортировка лучше - сортировка вставками или сортирорвка слиянием?')
print(result)

Вот отличие между ними:

* Сортировка вставками (Insertion sort) - это простой алгоритм, который работает медленно для больших массивов. Он наилучшим образом подходит для небольших отсортированных или почти отсортированных данных.
* Сортировка слиянием (Merge sort) - это более сложный алгоритм, который имеет линейную сложность в худшем случае. Он лучше подходит для больших массивов и может быть использован в случаях, когда данные не отсортированы.

Таким образом, выбор между ними зависит от размера и структуры данных:

* Если вы работаете с маленькими или почти отсортированными данными, сортировка вставками может быть более подходящей.
* Если вы работаете с большими массивами или не отсортированными данными, сортировка слиянием может быть более подходящей.

Обратите внимание, что существуют и другие алгоритмы сортировки, такие как Quick sort, Heap sort, Counting sort, которые могут быть использованы в зависимости от конкретной ситуации.


In [47]:
result = chain_for_sort.invoke('Какая сортировка лучше - быстрая сортировка или сортирорвка слиянием?')
print(result)

Русскоязычный ассистент здесь!

Ответ на ваш вопрос: выбор лучшей сортировки зависит от конкретной ситуации и требований. Обе сортировки (быстрая сортировка и сортировка слиянием) имеют свои преимущества и недостатки.

Быстрая сортировка (QuickSort) - это популярный алгоритм, который обычно работает за O(n log n) в среднем случае, но может быть медленнее при очень маленьких или очень больших входных данных. Ее преимущества включают:

* Огромственные временные complexities
* Низкая паметная потребность
* Удобство реализации

Недостатки быстрой сортировки - это потенциальные проблемы с пузырьками и медианами, которые могут привести к медленному выполнению.

Сортировка слиянием (MergeSort) - это стабильный алгоритм, который работает за O(n log n) в худшем случае. Ее преимущества включают:

* Высокую стабильность
* Низкая паметная потребность
* Удобство реализации

Недостатки сортировки слиянием - это то, что она может быть медленнее при маленьких входных данных и имеет потенциальные пробл

## Вывод

За исключением того, что модель ошибается в написании и формировании русских слов, она даёт правильные ответы на поставленные вопросы.